# Weekly Challenge 17

*Original URL* https://community.alteryx.com/t5/Weekly-Challenge/Challenge-17-Month-over-Month-Retention-Rate/td-p/36746 and [**My Alteryx Approach**](https://github.com/dsmdavid/Alteryx-Weekly-Challenge/tree/master/submitted/sub_Challenge%2317)

## Brief
A bank is looking to calculate customer retention rate month over month. The denominator in the calculation are all of the accounts open 24 months prior to the start of the month. For example, for June 2013, the denominator would be the total number of accounts open between June 1, 2011 through May 31, 2013. The numerator will be total number of accounts closed in June 2013 or between June 1, 2013 through June 30, 2013.

The objective is to create a batch macro that calculates the retention rate for May, June, July and August.


In [1]:
import pandas as pd
import os

## Approach I want to follow:
1. Read the input, truncate dates.  
1. Count accounts opened/closed per month, calculate running total and ratio as instructed.

## 1. Read and combine

In [2]:
#Read the files
os.chdir(os.path.join(os.getcwd(), '17_files'))

In [3]:
#Load the data:
input_df = pd.read_csv("./01_input.csv",
                       encoding="latin",
                       parse_dates=['Open Date', 'Close Date'])     


In [4]:
input_df

,RecordID,Open Date,Close Date
0,1,2013-04-03,2013-05-06
1,2,2013-04-14,NaT
2,3,2013-05-03,2013-07-18
3,4,2013-05-24,2013-06-12
4,5,2013-06-13,2013-07-10
5,6,2013-06-26,NaT
6,7,2013-07-04,NaT
7,8,2013-07-15,2013-08-09
8,9,2013-07-21,NaT
9,10,2013-08-13,NaT


In [5]:
# Truncate Dates to first of month
input_df['OpenMonth'] = input_df['Open Date'].apply(lambda x: x.replace(day =1))
input_df['CloseMonth'] = input_df['Close Date'].apply(lambda x: x.replace(day =1))

## 2. Count and summarize

In [6]:
# Summarize accounts by month open and month close
opened = input_df.groupby(by='OpenMonth')['RecordID'].count()
closed = input_df.groupby(by='CloseMonth')['RecordID'].count()

In [7]:
# Combine the two summarizes and calculate the running total, as well as the numerator, denominator and ratio as instructed
agg_df = pd.DataFrame([opened, closed]).T
agg_df.columns = ['Opened', 'Closed']
agg_df.fillna(0, inplace=True)
# Change in accounts that remain opened per month
agg_df['monthlyDelta'] = agg_df['Opened'] - agg_df['Closed']
# Total number of accounts opened at the end of the month
agg_df['Running'] = agg_df['monthlyDelta'].cumsum()
agg_df['AccountsEOPM'] = agg_df['Running'] - agg_df['monthlyDelta']
agg_df['ratio'] = agg_df['Closed'] / agg_df['AccountsEOPM']

In [8]:
solution = agg_df[['Opened', 'Closed','ratio']]
solution

,Opened,Closed,ratio
2013-04-01,2.0,0.0,NaN
2013-05-01,2.0,1.0,0.500000
2013-06-01,2.0,1.0,0.333333
2013-07-01,3.0,2.0,0.500000
2013-08-01,1.0,1.0,0.200000


I reckon I deviate slightly from the original challenge, that required a batch macro to process the given months, but this seems a simpler solution that achieves the same result.